In [1]:
from pathlib import Path

import numpy as np
from astropy.io import ascii
import matplotlib.pyplot as plt

from dask.distributed import Client
import dask.array
from dask.dataframe.utils import make_meta

from hats import read_hats
from hats.inspection import plot_pixels
from hats_import.catalog.file_readers import CsvReader
from hats_import.margin_cache.margin_cache_arguments import MarginCacheArguments
from hats_import.pipeline import ImportArguments, pipeline_with_client

import lsdb

from catalog_filtering import bandFilterLenient, contains_PM

print("Imported Libraries")

Imported Libraries


In [2]:
DES_HATS_DIR = Path("../../../shared/hats/catalogs/des/des_dr2")
GAIA_HATS_DIR = Path("../../../shared/hats/catalogs/gaia_dr3/gaia")

CATALOG_DIR = Path("../../catalogs")

GAIA_PM_NAME = "gaia_dr3_pm_greater_100"
GAIA_PM_DIR = CATALOG_DIR / GAIA_PM_NAME
GAIA_MARGIN_CACHE_NAME = "gaia_margin_cache_18_arcsec"
GAIA_MARGIN_CACHE_DIR = CATALOG_DIR / GAIA_MARGIN_CACHE_NAME

DES_LIGHT_NAME = "des_light"
DES_LIGHT_DIR = CATALOG_DIR / DES_LIGHT_NAME

XMATCH_NAME = "des_dr2_x_gaia_dr3"
XMATCH_DIR = CATALOG_DIR / XMATCH_NAME

print("Defined directories")

Defined directories


In [3]:
bandList = ['G','R','I','Z','Y']
class_star = None
spread_model = 0.05
magnitude_error = 0.05
check_flags = True
check_invalid_mags = True
query_string = bandFilterLenient(bandList,classStar=class_star,spreadModel=spread_model,magError=magnitude_error,flag=check_flags,invalidMags=check_invalid_mags)
des_cols = (
    [f'CLASS_STAR_{band}' for band in bandList] + 
    [f'FLAGS_{band}' for band in bandList] + 
    ['RA','DEC','COADD_OBJECT_ID'] + 
    [f'SPREAD_MODEL_{band}' for band in bandList] + 
    [f'WAVG_MAG_PSF_{band}' for band in bandList] + 
    [f'WAVG_MAGERR_PSF_{band}' for band in bandList]
)
pm_speed_min = 2000 #units are milliseconds per year
pm_speed_max = 10**5
max_neighbor_dist = 18
print("Defined filter variables")

Defined filter variables


# Obtaining old Gaia Catalog

In [ ]:
%%time
with Client(n_workers=4):
    lsdb.read_hats(
        path=GAIA_HATS_DIR,
        columns=['ra', 'pmra', 'dec', 'pmdec', 'source_id', 'parallax_over_error'],
    ).query('pmra*pmra + pmdec*pmdec > 100*100').to_hats(catalog_name = GAIA_PM_NAME, 
                                                        base_catalog_path = GAIA_PM_DIR)

print("gaia_pm successfully saved to /catalog")

## Obtaining Gaia Margin Cache

This cache ensures that we do not miss out on potential crossmatches of DES and Gaia at the border of pixels in the Gaia catalog. The margin threshold is defined by the maximum distance at which we seek object alignment for our algorithm: max_neighbor_dist.

In [ ]:
client = Client()

In [ ]:
margin_cache_args = MarginCacheArguments(
    input_catalog_path=GAIA_PM_DIR,
    output_path=CATALOG_DIR,
    margin_threshold=max_neighbor_dist,  # arcsec
    output_artifact_name=GAIA_MARGIN_CACHE_NAME,
)

pipeline_with_client(margin_cache_args, client)

In [ ]:
client.close()

In [12]:
%%time

with Client(n_workers=4) as client:
    lsdb.read_hats(
        DES_HATS_DIR,
        columns=des_cols,
    ).to_hats(catalog_name = DES_LIGHT_NAME, 
                base_catalog_path = DES_LIGHT_DIR)

CPU times: user 42.1 s, sys: 5.17 s, total: 47.2 s
Wall time: 5min 27s


In [4]:
des_dr2_light = lsdb.read_hats(DES_LIGHT_DIR)
gaia_dr3_pm_greater_100 = lsdb.read_hats(GAIA_PM_DIR, margin_cache=GAIA_MARGIN_CACHE_DIR)

des_dr2_light

,CLASS_STAR_G,CLASS_STAR_R,CLASS_STAR_I,CLASS_STAR_Z,CLASS_STAR_Y,FLAGS_G,FLAGS_R,FLAGS_I,FLAGS_Z,FLAGS_Y,RA,DEC,COADD_OBJECT_ID,SPREAD_MODEL_G,SPREAD_MODEL_R,SPREAD_MODEL_I,SPREAD_MODEL_Z,SPREAD_MODEL_Y,WAVG_MAG_PSF_G,WAVG_MAG_PSF_R,WAVG_MAG_PSF_I,WAVG_MAG_PSF_Z,WAVG_MAG_PSF_Y,WAVG_MAGERR_PSF_G,WAVG_MAGERR_PSF_R,WAVG_MAGERR_PSF_I,WAVG_MAGERR_PSF_Z,WAVG_MAGERR_PSF_Y
npartitions=1582,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 4, Pixel: 0",double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 5, Pixel: 8",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 743",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 1, Pixel: 47",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
gaia_dr3_pm_greater_100

,source_id,ra,dec,pmra,pmdec
npartitions=3917,,,,,
"Order: 2, Pixel: 0",int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 3, Pixel: 4",...,...,...,...,...
...,...,...,...,...,...
"Order: 4, Pixel: 3067",...,...,...,...,...
"Order: 3, Pixel: 767",...,...,...,...,...


In [5]:
xmatch = des_dr2_light.crossmatch(gaia_dr3_pm_greater_100,
    suffixes=("_des","_gaia"),
    n_neighbors=100, radius_arcsec=18,
)
xmatch

,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,FLAGS_G_des,FLAGS_R_des,FLAGS_I_des,FLAGS_Z_des,FLAGS_Y_des,RA_des,DEC_des,COADD_OBJECT_ID_des,SPREAD_MODEL_G_des,SPREAD_MODEL_R_des,SPREAD_MODEL_I_des,SPREAD_MODEL_Z_des,SPREAD_MODEL_Y_des,WAVG_MAG_PSF_G_des,WAVG_MAG_PSF_R_des,WAVG_MAG_PSF_I_des,WAVG_MAG_PSF_Z_des,WAVG_MAG_PSF_Y_des,WAVG_MAGERR_PSF_G_des,WAVG_MAGERR_PSF_R_des,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,source_id_gaia,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,_dist_arcsec
npartitions=1587,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 4, Pixel: 0",double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 5, Pixel: 8",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 752",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 767",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
with Client(n_workers=3):
    xmatch.to_hats(catalog_name = XMATCH_NAME,
        base_catalog_path = XMATCH_DIR
    )
print("Saved crossmatch to catalogs folder.")

2025-05-22 11:59:29,986 - distributed.worker.memory - WARNING - Worker is at 82% memory usage. Pausing worker.  Process memory: 8.13 GiB -- Worker memory limit: 9.90 GiB
2025-05-22 11:59:30,248 - distributed.worker.memory - WARNING - Worker is at 47% memory usage. Resuming worker. Process memory: 4.75 GiB -- Worker memory limit: 9.90 GiB
2025-05-22 11:59:41,386 - distributed.worker.memory - WARNING - Worker is at 84% memory usage. Pausing worker.  Process memory: 8.36 GiB -- Worker memory limit: 9.90 GiB
2025-05-22 11:59:41,902 - distributed.worker.memory - WARNING - Worker is at 49% memory usage. Resuming worker. Process memory: 4.92 GiB -- Worker memory limit: 9.90 GiB
2025-05-22 11:59:51,062 - distributed.worker.memory - WARNING - gc.collect() took 1.140s. This is usually a sign that some tasks handle too many Python objects at the same time. Rechunking the work into smaller tasks might help.
2025-05-22 11:59:51,063 - distributed.worker.memory - WARNING - Worker is at 90% memory usa

Saved crossmatch to catalogs folder.


In [8]:
des_gaia_xmatch = lsdb.read_hats(XMATCH_DIR, columns='all')

des_gaia_xmatch

,CLASS_STAR_G_des,CLASS_STAR_R_des,CLASS_STAR_I_des,CLASS_STAR_Z_des,CLASS_STAR_Y_des,FLAGS_G_des,FLAGS_R_des,FLAGS_I_des,FLAGS_Z_des,FLAGS_Y_des,RA_des,DEC_des,COADD_OBJECT_ID_des,SPREAD_MODEL_G_des,SPREAD_MODEL_R_des,SPREAD_MODEL_I_des,SPREAD_MODEL_Z_des,SPREAD_MODEL_Y_des,WAVG_MAG_PSF_G_des,WAVG_MAG_PSF_R_des,WAVG_MAG_PSF_I_des,WAVG_MAG_PSF_Z_des,WAVG_MAG_PSF_Y_des,WAVG_MAGERR_PSF_G_des,WAVG_MAGERR_PSF_R_des,WAVG_MAGERR_PSF_I_des,WAVG_MAGERR_PSF_Z_des,WAVG_MAGERR_PSF_Y_des,source_id_gaia,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia,_dist_arcsec
npartitions=1578,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Order: 4, Pixel: 0",double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],int16[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],int64[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow],double[pyarrow]
"Order: 5, Pixel: 8",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 743",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Order: 3, Pixel: 752",...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
des_gaia_xmatch.all_columns

['CLASS_STAR_G_des',
 'CLASS_STAR_R_des',
 'CLASS_STAR_I_des',
 'CLASS_STAR_Z_des',
 'CLASS_STAR_Y_des',
 'FLAGS_G_des',
 'FLAGS_R_des',
 'FLAGS_I_des',
 'FLAGS_Z_des',
 'FLAGS_Y_des',
 'RA_des',
 'DEC_des',
 'COADD_OBJECT_ID_des',
 'SPREAD_MODEL_G_des',
 'SPREAD_MODEL_R_des',
 'SPREAD_MODEL_I_des',
 'SPREAD_MODEL_Z_des',
 'SPREAD_MODEL_Y_des',
 'WAVG_MAG_PSF_G_des',
 'WAVG_MAG_PSF_R_des',
 'WAVG_MAG_PSF_I_des',
 'WAVG_MAG_PSF_Z_des',
 'WAVG_MAG_PSF_Y_des',
 'WAVG_MAGERR_PSF_G_des',
 'WAVG_MAGERR_PSF_R_des',
 'WAVG_MAGERR_PSF_I_des',
 'WAVG_MAGERR_PSF_Z_des',
 'WAVG_MAGERR_PSF_Y_des',
 'source_id_gaia',
 'ra_gaia',
 'dec_gaia',
 'pmra_gaia',
 'pmdec_gaia',
 '_dist_arcsec']

In [5]:
pm_filter_xmatch = des_gaia_xmatch.query(f'{pm_speed_max**2} >(pmra_gaia**2 + pmdec_gaia**2) > {pm_speed_min**2}')
with Client(n_workers=4):
    df = pm_filter_xmatch.compute()

df

,source_id_gaia,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia
_healpix_29,,,,,
1153482605725265461,2306965202564744064,1.383284,-37.367744,5633.438088,-2334.721273
1153482609302817738,2306965202564744064,1.383284,-37.367744,5633.438088,-2334.721273
...,...,...,...,...,...
3206298013938319236,6412596012146801152,331.076453,-56.793812,3981.976663,-2466.831815
3206298016903817602,6412596012146801152,331.076453,-56.793812,3981.976663,-2466.831815


In [6]:
df_no_dupes = df[~df['source_id_gaia'].duplicated(keep='first')]

gaia_ids = df_no_dupes['source_id_gaia']

#dropping because otherwise produces error when performing .apply below
df_no_dupes = df_no_dupes.drop('source_id_gaia', axis=1)

df_no_dupes

,ra_gaia,dec_gaia,pmra_gaia,pmdec_gaia
_healpix_29,,,,
1153482605725265461,1.383284,-37.367744,5633.438088,-2334.721273
1257518643390044839,33.079599,3.567385,-1762.405718,-1852.8711
2368327529620177120,53.567196,-49.890084,2360.592206,483.127504
2390050329526096144,62.611,-53.612997,-825.17937,-2415.577565
2405297220004419732,77.959937,-45.043813,6491.223339,-5708.61415
2423978658324676571,50.000344,-43.066553,3035.017316,726.964482
2450054482295873745,5.03561,-64.869617,1706.746855,1164.959443
2468500457058623057,32.622946,-50.820906,2125.416147,637.975043
2468500457058623057,32.624069,-50.820823,2168.886011,710.847727
